# Extreme Heat Days Calculator - Bahia State

Calculate extreme heat days using ERA5-Land LST data aligned to WorldPop 100m grid.

**Formula:** `Surface_Heat_Day = 1 if LST_daily_max > max(LST_abs, LST_rel)`

Where:
- `LST_rel = percentile_X{LST_daily_max for calendar_day ± 5 days over 30-year period}`
- Output: GeoTIFF aligned to WorldPop Brazil 100m grid

In [67]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Initialize Google Earth Engine
try:
    ee.Initialize(project='tl-cities')
    print("✅ Google Earth Engine initialized")
except Exception as e:
    print(f"❌ GEE initialization failed: {e}")

print("🌡️ Extreme Heat Days Calculator Ready")

✅ Google Earth Engine initialized
🌡️ Extreme Heat Days Calculator Ready


## Load Data Sources and Define Region

In [ ]:
# =============================================================================
# ROI (Region of Interest) Selection & Management
# =============================================================================

print("🎯 Setting up ROI selection options...")

# Create ROI selection widgets
roi_method = widgets.RadioButtons(
    options=['Use Test Areas', 'Draw ROI on Map', 'Upload Reference Raster'],
    value='Use Test Areas',
    description='ROI Method:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Test area selector
test_area_selector = widgets.Dropdown(
    options=[
        ('Salvador (5km x 5km)', 'salvador_small'),
        ('Salvador (10km x 10km)', 'salvador_medium'),
        ('Feira de Santana (5km x 5km)', 'feira_small'),
        ('Custom Small (2km x 2km)', 'custom_tiny')
    ],
    value='salvador_small',
    description='Test Area:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

file_upload = widgets.FileUpload(
    accept='.tif,.tiff,.geotiff',
    multiple=False,
    description='Upload GeoTIFF:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

roi_status = widgets.Output(layout=widgets.Layout(height='120px'))

# ROI management buttons
set_test_area_button = widgets.Button(
    description='🎯 Set Test Area',
    button_style='success',
    layout=widgets.Layout(width='140px')
)

set_roi_button = widgets.Button(
    description='📍 Set from Drawing',
    button_style='info',
    layout=widgets.Layout(width='140px')
)

show_roi_button = widgets.Button(
    description='👁️ Show ROI',
    button_style='warning',
    layout=widgets.Layout(width='100px')
)

clear_roi_button = widgets.Button(
    description='🗑️ Clear',
    button_style='',
    layout=widgets.Layout(width='80px')
)

# Global variables for ROI
analysis_geom = None
target_projection = None
target_scale = 100  # Default 100m

def setup_roi_map():
    """Create initial map for ROI selection"""
    roi_map = geemap.Map(center=[-12.0, -41.5], zoom=6)
    roi_map.add_basemap('SATELLITE')
    
    # Add example locations for reference
    example_cities = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point([-38.5014, -12.9714]), {'name': 'Salvador'}),
        ee.Feature(ee.Geometry.Point([-40.2108, -15.5989]), {'name': 'Feira de Santana'}),
        ee.Feature(ee.Geometry.Point([-39.0639, -14.8579]), {'name': 'Vitória da Conquista'})
    ])
    
    roi_map.addLayer(example_cities, {'color': 'yellow'}, 'Example Cities')
    
    return roi_map

def get_test_area_geometry(area_code):
    """Get predefined test area geometries"""
    if area_code == 'salvador_small':
        center_lon, center_lat = -38.5014, -12.9714
        buffer = 0.025  # ~2.5km
    elif area_code == 'salvador_medium':
        center_lon, center_lat = -38.5014, -12.9714
        buffer = 0.05   # ~5km
    elif area_code == 'feira_small':
        center_lon, center_lat = -40.2108, -15.5989
        buffer = 0.025  # ~2.5km
    elif area_code == 'custom_tiny':
        center_lon, center_lat = -38.5, -12.95
        buffer = 0.01   # ~1km
    else:
        center_lon, center_lat = -38.5014, -12.9714
        buffer = 0.025
    
    return ee.Geometry.Rectangle([
        center_lon - buffer,
        center_lat - buffer, 
        center_lon + buffer,
        center_lat + buffer
    ])

def handle_roi_method_change(change):
    """Handle ROI method selection"""
    with roi_status:
        clear_output(wait=True)
        
        if change['new'] == 'Use Test Areas':
            print("🎯 Test Areas Method Selected")
            print("Quick start with predefined areas around major cities.")
            test_area_selector.layout.display = 'block'
            set_test_area_button.layout.display = 'block'
            file_upload.layout.display = 'none'
            set_roi_button.layout.display = 'none'
            
        elif change['new'] == 'Draw ROI on Map':
            print("📍 Draw ROI Method Selected")
            print("Use the drawing tools on the map below to create custom areas.")
            test_area_selector.layout.display = 'none'
            set_test_area_button.layout.display = 'none'
            file_upload.layout.display = 'none'
            set_roi_button.layout.display = 'block'
            
        else:  # Upload Reference Raster
            print("📁 Upload Raster Method Selected")
            print("Upload a GeoTIFF to use its extent as the analysis area.")
            print("⚠️ Feature coming soon - currently creates test area")
            test_area_selector.layout.display = 'none'
            set_test_area_button.layout.display = 'none'
            file_upload.layout.display = 'block'
            set_roi_button.layout.display = 'none'

def set_test_area():
    """Set predefined test area"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        try:
            area_code = test_area_selector.value
            analysis_geom = get_test_area_geometry(area_code)
            target_projection = 'EPSG:4326'
            target_scale = 100
            
            # Get area info
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            area_name = [opt[0] for opt in test_area_selector.options if opt[1] == area_code][0]
            
            print(f"✅ Test area set: {area_name}")
            print(f"   Area: {area_km2:.1f} km²")
            print(f"   Projection: {target_projection}")
            print(f"   Resolution: {target_scale}m")
            print("Ready for analysis!")
            
            # Auto-show on map
            show_roi_on_map(quiet=True)
            
        except Exception as e:
            print(f"❌ Error setting test area: {e}")

def handle_file_upload(change):
    """Handle uploaded raster file"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        if file_upload.value:
            try:
                print("🔄 Processing uploaded raster...")
                print("⚠️ Full file processing not implemented yet")
                print("Creating example area around Salvador...")
                
                # Fallback to test area for now
                center_lon, center_lat = -38.5014, -12.9714
                buffer_size = 0.05
                
                analysis_geom = ee.Geometry.Rectangle([
                    center_lon - buffer_size,
                    center_lat - buffer_size, 
                    center_lon + buffer_size,
                    center_lat + buffer_size
                ])
                
                target_projection = 'EPSG:4326'
                target_scale = 100
                
                print(f"✅ ROI set to example area")
                print(f"   Projection: {target_projection}")
                print(f"   Resolution: {target_scale}m")
                print("Ready for analysis!")
                
            except Exception as e:
                print(f"❌ Error processing file: {e}")

def set_roi_from_drawing():
    """Get ROI from drawn geometry on map"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        try:
            # Try to get drawn geometry
            if hasattr(roi_map, 'draw_features') and roi_map.draw_features:
                drawn_feature = roi_map.draw_features[-1]
                geometry_data = drawn_feature['geometry']
                
                # Convert to EE geometry
                if geometry_data['type'] == 'Polygon':
                    coords = geometry_data['coordinates'][0]
                    analysis_geom = ee.Geometry.Polygon(coords)
                elif geometry_data['type'] == 'Rectangle':
                    coords = geometry_data['coordinates'][0]
                    analysis_geom = ee.Geometry.Rectangle(coords)
                else:
                    raise ValueError(f"Unsupported geometry type: {geometry_data['type']}")
                
                target_projection = 'EPSG:4326'
                target_scale = 100
                
                # Get area info
                area_km2 = analysis_geom.area().divide(1000000).getInfo()
                
                print(f"✅ ROI set from drawing")
                print(f"   Area: {area_km2:.1f} km²")
                print(f"   Projection: {target_projection}")
                print(f"   Resolution: {target_scale}m")
                print("Ready for analysis!")
                
                # Auto-show on analysis map
                show_roi_on_map(quiet=True)
                
            else:
                print("❌ No drawing found")
                print("Please draw a rectangle or polygon on the map first")
                
        except Exception as e:
            print(f"❌ Error setting ROI: {e}")
            print("Creating fallback test area...")
            
            # Fallback
            analysis_geom = ee.Geometry.Rectangle([-38.55, -12.95, -38.45, -12.90])
            target_projection = 'EPSG:4326' 
            target_scale = 100
            
            print("✅ Using fallback test area")

def show_roi_on_map(quiet=False):
    """Display current ROI on the analysis map"""
    if not quiet:
        with roi_status:
            clear_output(wait=True)
    
    if analysis_geom is None:
        if not quiet:
            print("❌ No ROI set yet. Please set an ROI first.")
        return
        
    try:
        # Add ROI to the analysis map (m)
        roi_style = {
            'color': 'red',
            'fillColor': 'red',
            'fillOpacity': 0.2,
            'width': 3,
            'opacity': 1.0
        }
        m.addLayer(analysis_geom, roi_style, 'Current ROI')
        
        # Center map on ROI
        m.centerObject(analysis_geom, 10)
        
        if not quiet:
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            print(f"✅ ROI displayed on analysis map (Area: {area_km2:.1f} km²)")
        
    except Exception as e:
        if not quiet:
            print(f"❌ Error showing ROI: {e}")

def clear_roi():
    """Clear current ROI"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        analysis_geom = None
        target_projection = None
        target_scale = 100
        
        print("✅ ROI cleared")
        print("Please select and set a new ROI for analysis")

# Connect event handlers
roi_method.observe(handle_roi_method_change, names='value')
file_upload.observe(handle_file_upload, names='value')
set_test_area_button.on_click(lambda b: set_test_area())
set_roi_button.on_click(lambda b: set_roi_from_drawing())
show_roi_button.on_click(lambda b: show_roi_on_map())
clear_roi_button.on_click(lambda b: clear_roi())

# Create ROI control interface
roi_buttons = widgets.HBox([
    set_test_area_button,
    set_roi_button,
    show_roi_button,
    clear_roi_button
])

roi_controls = widgets.VBox([
    widgets.HTML("<h4>🎯 Region of Interest (ROI) Selection</h4>"),
    roi_method,
    test_area_selector,
    file_upload,
    roi_buttons,
    roi_status
])

# Create and display ROI map
roi_map = setup_roi_map()

# Display interface
display(roi_controls)
display(roi_map)

# Initialize with default method
handle_roi_method_change({'new': 'Use Test Areas'})

print("✅ ROI selection interface ready!")
print("\n💡 Quick Start:")
print("1. Keep 'Salvador (5km x 5km)' selected")
print("2. Click '🎯 Set Test Area'")
print("3. Scroll down to run analysis")

# Load data sources
print("\nLoading data sources...")
try:
    era5_land = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    print("✅ ERA5-Land dataset loaded")
    
    worldpop = ee.ImageCollection("WorldPop/GP/100m/pop")
    brazil_pop = worldpop.filter(ee.Filter.eq('country', 'BRA')).sort('system:time_start', False).first()
    worldpop_proj = brazil_pop.projection()
    worldpop_crs = worldpop_proj.getInfo()['crs']
    print(f"✅ WorldPop reference loaded (CRS: {worldpop_crs})")
    
except Exception as e:
    print(f"❌ Data loading failed: {e}")
    worldpop_crs = 'EPSG:4326'

print("\n📝 Note: ERA5-Land covers land areas only")
print("   Coastal/water areas in ROI may show no data (this is normal)")

<function_calls>
<invoke name="TodoWrite">
<parameter name="todos">[{"content": "Verify full spatial coverage in results", "status": "completed", "activeForm": "Verified full spatial coverage - ERA5-Land land-only limitation confirmed"}, {"content": "Clean up debugging cells from notebook", "status": "completed", "activeForm": "Cleaned up debugging cells from notebook"}, {"content": "Incorporate ROI display and upload features into main interface", "status": "in_progress", "activeForm": "Incorporating ROI display and upload features into main interface"}]

## Extreme Heat Days Calculation Functions

In [ ]:
def create_buffered_roi(region_geom, buffer_distance=0.1):
    """
    Create a buffered ROI for data collection to ensure ERA5 coverage
    buffer_distance in degrees (~11km per 0.1 degree)
    """
    buffered_geom = region_geom.buffer(buffer_distance)
    return buffered_geom

def calculate_simple_extreme_heat_days(region_geom, year, lst_abs_threshold=40, percentile_threshold=95, 
                                      baseline_start='2015-01-01', baseline_end='2020-12-31'):
    """
    Simplified extreme heat days calculation - much faster and memory efficient
    Uses single percentile across entire baseline period (no day-of-year specificity)
    """
    print(f"   Calculating extreme heat days for {year} (Simplified Mode)...")
    
    # Create buffered ROI for data collection only
    buffer_km = 15  # 15km buffer
    buffer_degrees = buffer_km / 111.0  # Rough conversion: 1 degree ≈ 111km
    buffered_roi = create_buffered_roi(region_geom, buffer_degrees)
    
    print(f"   Using {buffer_km}km buffer for data collection, then clipping to original ROI")
    
    # Get baseline collection using buffered ROI for data extraction
    baseline_collection = era5_land.filterDate(baseline_start, baseline_end) \
                                   .filterBounds(buffered_roi) \
                                   .select(['temperature_2m'])
    
    baseline_count = baseline_collection.size().getInfo()
    print(f"   Baseline images found: {baseline_count}")
    
    # Convert to LST (Celsius) and clip to original ROI only
    baseline_lst = baseline_collection.map(lambda img: 
        img.select('temperature_2m').subtract(273.15).rename('LST').clip(region_geom)
    )
    
    # Calculate simple percentile across all days - clip to original ROI
    print(f"   Computing single {percentile_threshold}th percentile across baseline period...")
    lst_percentile = baseline_lst.select('LST').reduce(ee.Reducer.percentile([percentile_threshold])).clip(region_geom)
    
    # Get year collection using buffered ROI for data availability
    year_collection = era5_land.filterDate(f'{year}-01-01', f'{year}-12-31') \
                               .filterBounds(buffered_roi) \
                               .select(['temperature_2m'])
    
    year_count = year_collection.size().getInfo()
    print(f"   Target year images found: {year_count}")
    
    # Convert to LST (Celsius) - clip to original ROI only
    year_lst = year_collection.map(lambda img: 
        img.select('temperature_2m').subtract(273.15).rename('LST').clip(region_geom)
    )
    
    # Create absolute threshold image - original ROI only
    lst_abs = ee.Image.constant(lst_abs_threshold).rename('LST_abs').clip(region_geom)
    
    # Use max of absolute threshold and calculated percentile - original ROI only
    threshold = lst_abs.max(lst_percentile).rename('threshold').clip(region_geom)
    
    def classify_heat_day(img):
        """
        Classify each day as extreme heat day or not
        """
        lst_daily = img.select('LST')
        heat_day = lst_daily.gt(threshold).rename('heat_day')
        return heat_day.clip(region_geom).set('system:time_start', img.get('system:time_start'))
    
    # Apply classification to all days
    heat_days_collection = year_lst.map(classify_heat_day)
    
    # Sum to get total extreme heat days - original ROI only
    total_extreme_heat_days = heat_days_collection.sum().rename('extreme_heat_days').clip(region_geom)
    
    return total_extreme_heat_days

def calculate_lst_percentiles(region_geom, percentile_value=95, baseline_start='1991-01-01', baseline_end='2020-12-31'):
    """
    Calculate LST percentiles for each calendar day ± 5 days over baseline period
    Returns an image with 365 bands (one for each day of year)
    WARNING: This is memory intensive for large baseline periods
    """
    print(f"   Calculating {percentile_value}th percentile LST for each calendar day...")
    print("   ⚠️  This may use significant memory for long baseline periods")
    
    # Create buffered ROI for data collection only
    buffer_km = 15
    buffer_degrees = buffer_km / 111.0
    buffered_roi = create_buffered_roi(region_geom, buffer_degrees)
    
    # Get baseline period collection using buffered ROI
    baseline_collection = era5_land.filterDate(baseline_start, baseline_end) \
                                   .filterBounds(buffered_roi) \
                                   .select(['temperature_2m'])
    
    # Convert from Kelvin to Celsius - clip to original ROI only
    baseline_lst = baseline_collection.map(lambda img: 
        img.select('temperature_2m').subtract(273.15).rename('LST').clip(region_geom)
        .set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year'))
    )
    
    def calculate_doy_percentile(doy):
        """
        Calculate percentile for specific day of year ± 5 days
        """
        doy = ee.Number(doy)
        
        # Create window around target day (±5 days, wrapping around year)
        start_doy = doy.subtract(5)
        end_doy = doy.add(5)
        
        # Handle year wrapping
        winter_filter = ee.Filter.And(
            ee.Filter.gte('doy', start_doy),
            ee.Filter.lte('doy', end_doy)
        )
        
        # Handle case where window wraps around year end
        wrap_filter = ee.Filter.Or(
            ee.Filter.gte('doy', start_doy.add(365)),
            ee.Filter.lte('doy', end_doy.subtract(365))
        )
        
        day_filter = ee.Algorithms.If(
            start_doy.lt(0).Or(end_doy.gt(365)),
            wrap_filter,
            winter_filter
        )
        
        # Filter images for this day window
        day_images = baseline_lst.filter(day_filter)
        
        # Calculate percentile - clip to original ROI only
        percentile_img = day_images.select('LST').reduce(ee.Reducer.percentile([percentile_value])).clip(region_geom)
        
        # Create band name using server-side operations only
        band_name = ee.String('LST_p').cat(ee.Number(percentile_value).format()).cat('_doy_').cat(doy.format('%03d'))
        
        return percentile_img.rename([band_name])
    
    # Calculate for all days of year (1-365) - MEMORY INTENSIVE
    doy_list = ee.List.sequence(1, 365)
    percentile_images = doy_list.map(calculate_doy_percentile)
    
    # Convert to multi-band image - original ROI only
    percentile_stack = ee.Image(ee.ImageCollection(percentile_images).toBands()).clip(region_geom)
    
    return percentile_stack

def calculate_extreme_heat_days(region_geom, year, lst_abs_threshold=40, percentile_threshold=95, 
                               baseline_start='1991-01-01', baseline_end='2020-12-31'):
    """
    Full extreme heat days calculation with day-of-year specific percentiles
    Surface_Heat_Day = 1 if LST_daily_max > max(LST_abs, LST_rel)
    WARNING: This is memory intensive
    """
    print(f"   Calculating extreme heat days for {year} (Full Mode)...")
    print("   ⚠️  This may use significant memory - consider simplified mode for testing")
    
    # Create buffered ROI for data collection only
    buffer_km = 15
    buffer_degrees = buffer_km / 111.0
    buffered_roi = create_buffered_roi(region_geom, buffer_degrees)
    
    # Get year collection using buffered ROI
    year_collection = era5_land.filterDate(f'{year}-01-01', f'{year}-12-31') \
                               .filterBounds(buffered_roi) \
                               .select(['temperature_2m'])
    
    # Convert to LST (Celsius) - clip to original ROI only
    year_lst = year_collection.map(lambda img: 
        img.select('temperature_2m').subtract(273.15).rename('LST').clip(region_geom)
        .set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year'))
        .set('system:time_start', img.get('system:time_start'))
    )
    
    # Calculate relative thresholds (percentiles by day of year)
    print(f"     Computing {percentile_threshold}th percentile thresholds...")
    percentile_stack = calculate_lst_percentiles(region_geom, percentile_threshold, baseline_start, baseline_end)
    
    # Create absolute threshold image - original ROI only
    lst_abs = ee.Image.constant(lst_abs_threshold).rename('LST_abs').clip(region_geom)
    
    def classify_heat_day(img):
        """
        Classify each day as extreme heat day or not
        """
        doy = ee.Number(img.get('doy'))
        
        # Get the appropriate percentile threshold for this day of year
        band_name = ee.String('LST_p').cat(ee.Number(percentile_threshold).format()).cat('_doy_').cat(doy.format('%03d'))
        
        # Get percentile threshold for this day, with fallback for missing bands
        lst_rel = ee.Algorithms.If(
            percentile_stack.bandNames().contains(band_name),
            percentile_stack.select([band_name]),
            ee.Image.constant(lst_abs_threshold).clip(region_geom)  # Fallback to absolute threshold
        )
        lst_rel = ee.Image(lst_rel).rename('LST_rel').clip(region_geom)
        
        # Calculate max(LST_abs, LST_rel) - original ROI only
        threshold = lst_abs.max(lst_rel).rename('threshold').clip(region_geom)
        
        # LST daily max (using temperature_2m as proxy)
        lst_daily_max = img.select('LST')
        
        # Surface_Heat_Day = 1 if LST_daily_max > threshold - original ROI only
        heat_day = lst_daily_max.gt(threshold).rename('heat_day').clip(region_geom)
        
        return heat_day.set('system:time_start', img.get('system:time_start'))
    
    # Apply classification to all days
    heat_days_collection = year_lst.map(classify_heat_day)
    
    # Sum to get total extreme heat days - original ROI only
    total_extreme_heat_days = heat_days_collection.sum().rename('extreme_heat_days').clip(region_geom)
    
    return total_extreme_heat_days

def align_to_target_grid(image, region_geom, target_crs='EPSG:4326', target_scale=100):
    """
    Align image to target grid by resampling
    """
    print(f"   Reprojecting to {target_crs} at {target_scale}m resolution...")
    
    # First ensure image is properly clipped to exact ROI
    clipped_image = image.clip(region_geom)
    
    # Use bilinear resampling for smooth coverage
    aligned_image = clipped_image.resample('bilinear').reproject(
        crs=target_crs,
        scale=target_scale
    )
    
    # Ensure final result is properly bounded
    final_result = aligned_image.clip(region_geom)
    
    return final_result

def parse_baseline_period(period_string):
    """Parse baseline period string to start/end dates"""
    if '2015-2020' in period_string:
        return '2015-01-01', '2020-12-31'
    elif '2010-2020' in period_string:
        return '2010-01-01', '2020-12-31'
    elif '1991-2020' in period_string:
        return '1991-01-01', '2020-12-31'
    else:
        return '2015-01-01', '2020-12-31'  # Default

print("✅ Extreme heat days calculation functions ready")

## Visualization and Export Functions

In [70]:
def create_heat_visualization():
    """
    Create visualization parameters for extreme heat days
    """
    # Heat-focused color palette (yellow to red)
    heat_palette = ['#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', 
                   '#fc4e2a', '#e31a1c', '#bd0026', '#800026']
    
    return {
        'palette': heat_palette,
        'min': 0,
        'max': 100  # Days per year - will be adjusted dynamically
    }

def export_to_drive(image, region_geom, filename, target_crs='EPSG:4326', target_scale=100, folder='GEE_Exports'):
    """
    Export image to Google Drive as GeoTIFF
    """
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder=folder,
        fileNamePrefix=filename,
        region=region_geom,
        scale=target_scale,
        crs=target_crs,
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    
    task.start()
    return task

def display_on_map(m, image, region_geom, layer_name, show_region=True):
    """
    Display results on interactive map with memory-efficient stats calculation
    """
    try:
        # Use larger scale for stats calculation to reduce memory usage
        stats_scale = max(1000, target_scale * 10)  # At least 1km resolution for stats
        
        # Get statistics for visualization scaling
        stats = image.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region_geom,
            scale=stats_scale,
            maxPixels=1e6,  # Reduced pixel limit
            bestEffort=True
        )
        
        stats_info = stats.getInfo()
        min_val = stats_info.get('extreme_heat_days_min', 0) or 0
        max_val = stats_info.get('extreme_heat_days_max', 50) or 50
        
        print(f"Data range: {min_val:.0f} to {max_val:.0f} extreme heat days per year")
        
        # Create visualization parameters
        vis_params = create_heat_visualization()
        vis_params['min'] = min_val
        vis_params['max'] = max_val
        
        # Add layer to map
        m.addLayer(image, vis_params, layer_name)
        
        # Add region boundary
        if show_region:
            region_style = {
                'color': 'blue',
                'fillColor': '00000000',
                'width': 2,
                'opacity': 1.0
            }
            m.addLayer(region_geom, region_style, 'Analysis ROI')
        
        # Center on region
        m.centerObject(region_geom, 10)  # Higher zoom for smaller ROI
        
        return min_val, max_val
        
    except Exception as e:
        print(f"⚠️ Could not calculate stats for visualization: {e}")
        print("Using default visualization parameters...")
        
        # Use default parameters if stats fail
        vis_params = create_heat_visualization()
        vis_params['min'] = 0
        vis_params['max'] = 50
        
        # Add layer to map
        m.addLayer(image, vis_params, layer_name)
        
        # Add region boundary
        if show_region:
            region_style = {
                'color': 'blue',
                'fillColor': '00000000',
                'width': 2,
                'opacity': 1.0
            }
            m.addLayer(region_geom, region_style, 'Analysis ROI')
        
        # Center on region
        m.centerObject(region_geom, 10)
        
        return 0, 50

print("✅ Visualization and export functions updated for ROI processing")

✅ Visualization and export functions updated for ROI processing


## Interactive Interface

In [71]:
# Create interactive map
m = geemap.Map(center=[-12.0, -41.5], zoom=6)  # Centered on Bahia

# Create control widgets
year_selector = widgets.Dropdown(
    options=list(range(2020, 2024)),  # Reduced range for testing
    value=2023,
    description='Analysis Year:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='200px')
)

# Add simplified mode option
analysis_mode = widgets.RadioButtons(
    options=['Simplified (Fast)', 'Full Percentiles (Slow)'],
    value='Simplified (Fast)',
    description='Analysis Mode:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='250px')
)

baseline_period = widgets.Dropdown(
    options=['2015-2020 (5 years)', '2010-2020 (10 years)', '1991-2020 (30 years)'],
    value='2015-2020 (5 years)',
    description='Baseline Period:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='250px')
)

lst_abs_slider = widgets.FloatSlider(
    value=40.0,
    min=35.0,
    max=50.0,
    step=1.0,
    description='LST_abs (°C):',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

percentile_slider = widgets.IntSlider(
    value=95,
    min=90,
    max=99,
    step=1,
    description='Percentile:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

calculate_button = widgets.Button(
    description='🔥 Calculate Heat Days',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='35px')
)

export_button = widgets.Button(
    description='📁 Export to Drive',
    button_style='success',
    layout=widgets.Layout(width='200px', height='35px'),
    disabled=True
)

status_output = widgets.Output(layout=widgets.Layout(height='200px'))

# Global variables
current_result = None
export_task = None

print("✅ User interface widgets created with memory optimization options")

✅ User interface widgets created with memory optimization options


## Main Analysis Functions

In [72]:
def run_heat_analysis():
    """
    Main function to calculate extreme heat days with memory optimization
    """
    global current_result
    
    with status_output:
        clear_output(wait=True)
        
        # Check if ROI has been set
        if analysis_geom is None:
            print("❌ Please set an ROI first!")
            print("1. Select a test area and click 'Set Test Area'")
            print("2. OR draw an area on the ROI map, then click 'Set ROI from Drawing'")
            print("3. OR upload a reference raster file")
            return
        
        # Get parameters
        year = year_selector.value
        mode = analysis_mode.value
        baseline_period_str = baseline_period.value
        lst_abs = lst_abs_slider.value
        percentile = percentile_slider.value
        
        # Parse baseline period
        baseline_start, baseline_end = parse_baseline_period(baseline_period_str)
        
        # Get ROI area for reporting
        try:
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            area_text = f"({area_km2:.1f} km²)"
        except:
            area_text = ""
        
        print(f"🔄 Calculating Extreme Heat Days for Selected ROI {area_text}")
        print(f"   Analysis mode: {mode}")
        print(f"   Analysis year: {year}")
        print(f"   Baseline period: {baseline_start} to {baseline_end}")
        print(f"   LST absolute threshold: {lst_abs}°C")
        print(f"   LST relative threshold: {percentile}th percentile")
        print(f"   Resolution: {target_scale}m")
        
        if mode == 'Simplified (Fast)':
            print(f"   Formula: Heat_Day = 1 if LST_max > max({lst_abs}°C, overall_p{percentile})")
        else:
            print(f"   Formula: Heat_Day = 1 if LST_max > max({lst_abs}°C, daily_p{percentile})")
        
        try:
            # Choose calculation method based on mode
            if mode == 'Simplified (Fast)':
                print("   🚀 Using simplified calculation (single percentile threshold)...")
                extreme_heat_result = calculate_simple_extreme_heat_days(
                    region_geom=analysis_geom,
                    year=year,
                    lst_abs_threshold=lst_abs,
                    percentile_threshold=percentile,
                    baseline_start=baseline_start,
                    baseline_end=baseline_end
                )
            else:
                print("   🐌 Using full calculation (day-specific percentiles)...")
                extreme_heat_result = calculate_extreme_heat_days(
                    region_geom=analysis_geom,
                    year=year,
                    lst_abs_threshold=lst_abs,
                    percentile_threshold=percentile,
                    baseline_start=baseline_start,
                    baseline_end=baseline_end
                )
            
            print(f"   Aligning to target grid ({target_scale}m resolution)...")
            # Align to target grid
            aligned_result = align_to_target_grid(
                extreme_heat_result, 
                analysis_geom, 
                target_projection or 'EPSG:4326',
                target_scale
            )
            
            # Convert to float for export compatibility
            current_result = aligned_result.toFloat().rename('extreme_heat_days')
            
            print(f"   Displaying results on map...")
            # Display on map with memory-efficient visualization
            min_val, max_val = display_on_map(
                m, 
                current_result, 
                analysis_geom, 
                f'Heat Days {year}',
                show_region=True
            )
            
            # Enable export button
            export_button.disabled = False
            
            print(f"✅ Analysis complete!")
            print(f"   Range: {min_val:.0f} to {max_val:.0f} extreme heat days per year")
            print(f"   Grid: {target_scale}m resolution")
            print(f"   ROI area: ~{area_km2:.1f} km²")
            print(f"   Mode: {mode}")
            print(f"   Ready for export to GeoTIFF")
            
        except Exception as e:
            print(f"❌ Error in analysis: {str(e)}")
            import traceback
            print(f"   Details: {traceback.format_exc()}")
            
            if "memory limit" in str(e).lower() or "computation timed out" in str(e).lower():
                print("\n💡 Memory optimization suggestions:")
                print("• Try 'Simplified (Fast)' mode")
                print("• Use shorter baseline period (2015-2020)")
                print("• Use smaller ROI area")
                print("• Try higher resolution (e.g., 500m scale)")

def export_results():
    """
    Export results to Google Drive
    """
    global export_task
    
    with status_output:
        if current_result is None:
            print("⚠️ Please run analysis first before exporting")
            return
            
        if analysis_geom is None:
            print("⚠️ No ROI defined for export")
            return
        
        # Get parameters for filename
        year = year_selector.value
        mode = analysis_mode.value
        lst_abs = lst_abs_slider.value
        percentile = percentile_slider.value
        
        # Create descriptive filename
        mode_suffix = 'simple' if 'Simplified' in mode else 'full'
        try:
            area_km2 = int(analysis_geom.area().divide(1000000).getInfo())
            filename = f'heat_days_{year}_{mode_suffix}_abs{int(lst_abs)}_p{percentile}_{area_km2}km2_{target_scale}m'
        except:
            filename = f'heat_days_{year}_{mode_suffix}_abs{int(lst_abs)}_p{percentile}_{target_scale}m'
        
        print(f"🔄 Exporting to Google Drive...")
        print(f"   Filename: {filename}.tif")
        print(f"   Mode: {mode}")
        print(f"   Resolution: {target_scale}m")
        print(f"   Projection: {target_projection}")
        print(f"   Format: GeoTIFF")
        
        try:
            export_task = export_to_drive(
                current_result,
                analysis_geom,
                filename,
                target_projection,
                target_scale
            )
            
            print(f"✅ Export task started successfully!")
            print(f"   Task ID: {export_task.id}")
            print(f"   Check Google Earth Engine Tasks tab for progress")
            print(f"   File will appear in your Google Drive/GEE_Exports folder")
            
        except Exception as e:
            print(f"❌ Export failed: {str(e)}")

# Connect event handlers
def on_calculate_click(b):
    run_heat_analysis()

def on_export_click(b):
    export_results()

calculate_button.on_click(on_calculate_click)
export_button.on_click(on_export_click)

print("✅ Analysis functions updated with memory optimization")

✅ Analysis functions updated with memory optimization


## Display Interface

In [73]:
# Create control panel layout
controls_row1 = widgets.HBox([
    year_selector,
    analysis_mode,
    baseline_period
])

controls_row2 = widgets.HBox([
    lst_abs_slider,
    percentile_slider
])

controls_row3 = widgets.HBox([
    calculate_button,
    export_button
])

control_panel = widgets.VBox([
    widgets.HTML("<h3>🌡️ Extreme Heat Days Calculator - Custom ROI</h3>"),
    widgets.HTML("<p><b>Calculate extreme heat days using ERA5-Land LST for custom regions</b></p>"),
    widgets.HTML("<div style='background-color: #d1ecf1; padding: 10px; border-radius: 5px; margin: 10px 0;'>"
                "<strong>🚀 Memory Optimized:</strong><br>"
                "• <strong>Simplified (Fast):</strong> Single percentile threshold - much faster<br>"
                "• <strong>Full Percentiles (Slow):</strong> Day-specific thresholds - more accurate but memory intensive</div>"),
    widgets.HTML("<p><code>Simplified: Heat_Day = 1 if LST_daily_max > max(LST_abs, overall_percentile)</code></p>"),
    widgets.HTML("<p><code>Full: Heat_Day = 1 if LST_daily_max > max(LST_abs, daily_percentile)</code></p>"),
    widgets.HTML("<div style='background-color: #fff3cd; padding: 10px; border-radius: 5px; margin: 10px 0;'>"
                "<strong>⚠️ Important:</strong> Make sure you've set an ROI first using the ROI selection interface above!</div>"),
    controls_row1,
    controls_row2,
    controls_row3,
    status_output
])

# Display the complete interface
display(control_panel)
display(m)

print("\n🎯 Memory-Optimized Extreme Heat Days Calculator Ready!")
print("\n📋 Workflow:")
print("1. 🎯 Set ROI: Use the ROI selection interface above")
print("2. ⚙️ Configure: Select analysis parameters")
print("3. 🔥 Calculate: Click 'Calculate Heat Days' to run analysis")
print("4. 👁️ Review: Check results on the map")
print("5. 📁 Export: Click 'Export to Drive' to download GeoTIFF")

print("\n🚀 Memory Optimization Features:")
print("• Simplified Mode: Single percentile (fast, memory-efficient)")
print("• Full Mode: Day-specific percentiles (accurate, memory intensive)")
print("• Flexible baseline periods: 5, 10, or 30 years")
print("• Small test ROI areas for quick testing")
print("• Efficient stats calculation for visualization")

print("\n💡 Performance Recommendations:")
print("• Start with 'Simplified (Fast)' mode")
print("• Use '2015-2020 (5 years)' baseline for testing")
print("• Use small test areas (Salvador 5km x 5km)")
print("• Switch to 'Full Percentiles' only for final results")

print("\n📊 Output: Count of extreme heat days per year per pixel")

Map(center=[-12.0, -41.5], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…


🎯 Memory-Optimized Extreme Heat Days Calculator Ready!

📋 Workflow:
1. 🎯 Set ROI: Use the ROI selection interface above
2. ⚙️ Configure: Select analysis parameters
3. 🔥 Calculate: Click 'Calculate Heat Days' to run analysis
4. 👁️ Review: Check results on the map
5. 📁 Export: Click 'Export to Drive' to download GeoTIFF

🚀 Memory Optimization Features:
• Simplified Mode: Single percentile (fast, memory-efficient)
• Full Mode: Day-specific percentiles (accurate, memory intensive)
• Flexible baseline periods: 5, 10, or 30 years
• Small test ROI areas for quick testing
• Efficient stats calculation for visualization

💡 Performance Recommendations:
• Start with 'Simplified (Fast)' mode
• Use '2015-2020 (5 years)' baseline for testing
• Use small test areas (Salvador 5km x 5km)
• Switch to 'Full Percentiles' only for final results

📊 Output: Count of extreme heat days per year per pixel


In [74]:
# Check if the analysis completed and what variables exist
print("=== DEBUGGING ANALYSIS STATUS ===")
print(f"current_result exists: {'current_result' in globals()}")
print(f"export_button disabled: {export_button.disabled}")

if 'current_result' in globals() and current_result is not None:
  print(f"current_result type: {type(current_result)}")
  print("✅ Analysis appears to have completed")
else:
  print("❌ Analysis did not complete or current_result is None")

# Check map layers
print(f"\n=== MAP LAYERS ===")
if hasattr(m, '_layers'):
  print(f"Number of map layers: {len(m._layers) if m._layers else 0}")
  for i, layer in enumerate(m._layers or []):
      print(f"Layer {i}: {getattr(layer, 'name', 'Unknown')}")
else:
  print("No map layers found")

=== DEBUGGING ANALYSIS STATUS ===
current_result exists: True
export_button disabled: False
current_result type: <class 'ee.image.Image'>
✅ Analysis appears to have completed

=== MAP LAYERS ===
No map layers found


In [ ]:
# Results Display and Management
print("📊 Analysis Results Management")
print("Use this cell to check current analysis status and manage results")

if 'current_result' in globals() and current_result is not None:
    print("\n✅ Analysis result available")
    
    if 'analysis_geom' in globals() and analysis_geom is not None:
        try:
            # Show ROI info
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            print(f"   ROI area: {area_km2:.1f} km²")
            
            # Quick data check
            test_stats = current_result.reduceRegion(
                reducer=ee.Reducer.count().combine(ee.Reducer.minMax(), sharedInputs=True),
                geometry=analysis_geom,
                scale=1000,
                maxPixels=1e6,
                bestEffort=True
            ).getInfo()
            
            pixel_count = test_stats.get('extreme_heat_days_count', 0)
            min_val = test_stats.get('extreme_heat_days_min', None)
            max_val = test_stats.get('extreme_heat_days_max', None)
            
            print(f"   Data coverage: {pixel_count} pixels")
            print(f"   Value range: {min_val} to {max_val} heat days")
            
            if pixel_count == 0:
                print("   ⚠️ No data found - this may be normal for water areas")
                print("   💡 Try a more inland ROI if you need land coverage")
            else:
                print("   ✅ Ready for export!")
                
        except Exception as e:
            print(f"   Status check failed: {e}")
    else:
        print("   ⚠️ No ROI defined")
        
else:
    print("\n⭕ No analysis result found")
    print("   Please run the analysis using the interface above")

print("\n🗺️ Map Layers:")
print("• Red boundary = Current ROI")  
print("• Heat colors = Extreme heat days per year")
print("• Use map layer controls to toggle visibility")